#### Začneme s jednoduchým, ale velmi užitečným projektem: blikající LED s rozšířením na sekvenční blikání více LED. Použijeme jazyk TinyGo (Go pro mikrokontroléry) a platformu Arduino Uno R3.

#### 🔧 Co budeš potřebovat:

- Arduino Uno R3
- 3× LED dioda
- 3× rezistor (např. 220–330 Ω)
- Propojovací vodiče
- Breadboard

🔌 Schéma zapojení:

| Arduino Pin | Komponenta                        |
|-------------|-----------------------------------|
| D2          | LED1 přes rezistor do GND         |
| D3          | LED2 přes rezistor do GND         |
| D4          | LED3 přes rezistor do GND         |

*Pozn.*: Každá LED má **katodu** (kratší nožičku) připojenou přes rezistor do GND a **anodu** (delší nožičku) do příslušného digitálního pinu.

#### 🧠 Krok za krokem:

1. Zapojení
Zapoj LED diody podle tabulky výše.
Každou LED připoj přes rezistor do GND.
Zkontroluj, že máš správně orientované LED (anoda = delší noha = k Arduinu).
2. Instalace TinyGo a GO - vyhledejte online

In [ ]:
package main

import (
"machine"
"time"
)

var leds = []machine.Pin{machine.D2, machine.D3, machine.D4}

func main() {
// Nastavení pinů jako výstup
for _, led := range leds {
led.Configure(machine.PinConfig{Mode: machine.PinOutput})
}

for {
// Sekvenční blikání
for _, led := range leds {
led.High()
time.Sleep(300 * time.Millisecond)
led.Low()
}
}
}


In [ ]:
tinygo build -o blink.hex -target=arduino ./main.go

In [ ]:
# zjisti porty - lze zkontrolovat napři. i v arduino IDE softwaru

ls /dev/ttyACM*   # Linux
ls /dev/cu.usbmodem*  # macOS

In [ ]:
avrdude -v -p atmega328p -c arduino -P /dev/ttyACM0 -b 115200 -D -U flash:w:blink.hex:i

### 🔐 Projekt: Bezpečnostní zámek s tlačítky
#### 🧰 Co budeš potřebovat:

- Arduino Uno R3
- 3× tlačítko (spínač)
- 1× LED dioda
- 1× rezistor (220–330 Ω) pro LED
- 3× rezistor (10kΩ) pro pull-down na tlačítka
- Propojovací vodiče
- Breadboard

### 🔌 Schéma zapojení

| Arduino Pin | Komponenta                          |
|-------------|-------------------------------------|
| D2          | Tlačítko 1 (přes 10kΩ do GND)       |
| D3          | Tlačítko 2 (přes 10kΩ do GND)       |
| D4          | Tlačítko 3 (přes 10kΩ do GND)       |
| D5          | LED (přes 220Ω do GND)              |

#### 🧠 Logika zámku:

- Správná sekvence: např. 1 → 2 → 3
- Pokud uživatel zadá špatné pořadí, sekvence se resetuje.
- Po správném zadání se rozsvítí LED.

In [ ]:
package main

import (
"machine"
"time"
)

var (
buttons = []machine.Pin{machine.D2, machine.D3, machine.D4}
led     = machine.D5

// Správná sekvence: 0 → 1 → 2 (odpovídá D2 → D3 → D4)
code       = []int{0, 1, 2}
codeIndex  = 0
debounceMs = 200
)

func main() {
// Nastavení pinů
for _, btn := range buttons {
btn.Configure(machine.PinConfig{Mode: machine.PinInput})
}
led.Configure(machine.PinConfig{Mode: machine.PinOutput})

for {
for i, btn := range buttons {
if btn.Get() {
time.Sleep(debounceMs * time.Millisecond) // debounce
if btn.Get() {
if i == code[codeIndex] {
codeIndex++
if codeIndex == len(code) {
led.High() // správná sekvence
}
} else {
codeIndex = 0
led.Low() // reset
}
// čekej na uvolnění tlačítka
for btn.Get() {
time.Sleep(10 * time.Millisecond)
}
}
}
}
}
}


In [ ]:
tinygo build -o lock.hex -target=arduino ./main.go

avrdude -v -p atmega328p -c arduino -P /dev/ttyACM0 -b 115200 -D -U flash:w:lock.hex:i

### 🧪 Testování:

1. Stiskni tlačítka ve správném pořadí (např. D2 → D3 → D4).
2. LED se rozsvítí.
3. Pokud uděláš chybu, LED zhasne a musíš začít znovu.

### 💡 LED stmívač s potenciometrem (PWM řízení jasu)

Pomocí analogového vstupu z potenciometru budeme řídit jas LED diody pomocí **PWM** (Pulse Width Modulation).

#### 🧰 Co budeš potřebovat:

- Arduino Uno R3
- 1× LED dioda
- 1× rezistor (220–330 Ω)
- 1× potenciometr (např. 10kΩ)
- Propojovací vodiče
- Breadboard

### 🔌 Schéma zapojení

| Arduino Pin  | Komponenta                          |
|--------------|-------------------------------------|
| A0           | Střední vývod potenciometru         |
| 5V           | Jeden krajní vývod potenciometru    |
| GND          | Druhý krajní vývod potenciometru    |
| D5 (PWM)     | LED přes rezistor do GND            |

*Pozn.*: 
- LED: Anoda (delší noha) na D5 přes rezistor, katoda na GND
- Potenciometr: krajní vývody mezi 5V a GND, střední vývod na A0